# Convolutional Neural Network Applied on Cucumber9 Dataset
---

Descriptions: (from cifar10 description):

Loaded in this way, each of the batch files contains a dictionary with the following elements:
data -- a 10000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.
labels -- a list of 10000 numbers in the range 0-9. The number at index i indicates the label of the ith image in the array data.

The dataset contains another file, called batches.meta. It too contains a Python dictionary object. It has the following entries:
label_names -- a 10-element list which gives meaningful names to the numeric labels in the labels array described above. For example, label_names[0] == "airplane", label_names[1] == "automobile", etc.

### Import

In [1]:
import tensorflow as tf
import numpy as np
import cPickle

### Import all files

In [14]:
with open('./inputs/data_batch_1') as f:
    batch1 = cPickle.load(f)
with open('./inputs/data_batch_2') as f:
    batch2 = cPickle.load(f)
with open('./inputs/data_batch_3') as f:
    batch3 = cPickle.load(f)
with open('./inputs/data_batch_4') as f:
    batch4 = cPickle.load(f)
with open('./inputs/data_batch_5') as f:
    batch5 = cPickle.load(f)
with open('./inputs/batches.meta') as f:
    meta = cPickle.load(f)

In [12]:
batch1['data'].shape

(1485, 1024)

In [13]:
1485/3

495